In [7]:
import jsonlines
import whisper
import os
import torch
from tqdm import tqdm
import logging
import shutil
from pydub import AudioSegment

In [8]:
origin = [] 
data = []
with open("/data/ruiqi.yan/data/understanding/gk_listening/gk_after_asr.jsonl", 'r', encoding='utf-8') as f:
    for line in jsonlines.Reader(f):
        origin.append(line)

In [9]:
len(origin)

303

In [10]:
def save_wav(item, id, source_dir, target_dir):
    source_path1 = os.path.join("/data/ruiqi.yan/data/understanding/gk_listening", item['source_wav'])
    source_path2 = os.path.join(source_dir, str(item['id']) + '.wav')
    
    audio1 = AudioSegment.from_wav(source_path1)
    audio2 = AudioSegment.from_wav(source_path2)
    combined = audio1 + audio2

    target_directory = target_dir
    target_path = os.path.join(target_directory, "audio_" + str(id).zfill(4) + ".wav")
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
        
    combined.export(target_path, format="wav")

In [11]:
source_dir = "/data/ruiqi.yan/data/eval/gaokao/gaokao"
target_dir = "/data/ruiqi.yan/data/final/gaokao_test/audio"
for i, item in enumerate(tqdm(origin)):
    id = i + 1
    save_wav(item, id, source_dir, target_dir)
    txt_name = item['source_wav'].split('/')[-1].replace('wav', 'txt')
    with open(os.path.join("/data/ruiqi.yan/data/understanding/gk_listening/combine_conv_audio_text_short", txt_name)) as f:
        for line in f:
            conv = line
    new = {"id": id,
           "source_wav": os.path.join("audio/", "audio_" + str(id).zfill(4) + ".wav"),
           "source_text": conv.strip() + ' ' + item['source_text'],
           "target_text": item['target_text']
           }
    data.append(new)

  0%|          | 0/303 [00:00<?, ?it/s]

100%|██████████| 303/303 [00:02<00:00, 124.01it/s]


In [12]:
with jsonlines.open("/data/ruiqi.yan/data/final/gaokao_test/test.jsonl", mode='w') as writer:
    for item in data:
        writer.write(item)